In [ ]:
!pip install -q datasets accelerate evaluate trl accelerate bitsandbytes peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install -U datasets transformers pyarrow


In [ ]:
from huggingface_hub import login

# Paste your HF token when prompted (required to access Mistral 7B Instruct v2)

login()

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

model_id = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(model_id)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config)
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
import json
from datasets import Dataset

jsonl_path = "/content/100 samples.jsonl"

data = []
with open(jsonl_path, 'r', encoding='utf-8') as f:
    for line in f:
        line = line.strip()
        if line:  # ✅ skip empty lines
            data.append(json.loads(line))

dataset = Dataset.from_list(data)




from datasets import Dataset, DatasetDict
import random

dataset = dataset.shuffle(seed=42)


split_idx = int(0.95 * len(dataset))


train_dataset = dataset.select(range(split_idx))
test_dataset = dataset.select(range(split_idx, len(dataset)))

dataset = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})

print(dataset)


In [ ]:
def format_and_tokenize(example, tokenizer):
    formatted_text = tokenizer.apply_chat_template(
        example["messages"],
        tokenize=False,
        add_generation_prompt=True
    )
    return {"text": formatted_text}
# Set globally ONCE
tokenizer.pad_token = tokenizer.eos_token

processed_train = dataset["train"].map(
    format_and_tokenize,
    fn_kwargs={"tokenizer": tokenizer},
    remove_columns=["messages"],
    num_proc=4
)

processed_test = dataset["test"].map(
    format_and_tokenize,
    fn_kwargs={"tokenizer": tokenizer},
    remove_columns=["messages"],
    num_proc=4
)


In [ ]:
processed_test[2]

In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,  # rank
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="mistral-7b-hr-finetuned",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-5,
    num_train_epochs=3,
    bf16=True,
    logging_steps=10,
    save_steps=1,
    save_total_limit=2,
    report_to="tensorboard"
)

# === 5️⃣ SFTTrainer ===
trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_dataset,
    args=training_args
)

# === 6️⃣ Train ===
trainer.train()
